In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Path to folders containing images for each class
class1_folder = 'C://Users/HP/test/Desktop/logdata4/normal'
class2_folder = 'C://Users/HP/test/Desktop/logdata4/osteoarthriticS'

# Hyperparameters
input_shape = (224, 224, 3)
latent_dim = 128
epochs = 20
batch_size = 32
learning_rate = 0.001

# Load and preprocess images
def load_and_preprocess_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=input_shape[:2])
        img_array = img_to_array(img)
        img_array /= 255.0  # Normalize
        images.append(img_array)
        labels.append(filename)
    return np.array(images), labels

class1_images, class1_labels = load_and_preprocess_images(class1_folder)
class2_images, class2_labels = load_and_preprocess_images(class2_folder)

# Combine images and labels
images = np.vstack([class1_images, class2_images])
labels = np.concatenate([class1_labels, class2_labels])

# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Build the autoencoder
input_layer = Input(shape=input_shape)
encoder = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
encoder = MaxPooling2D((2, 2), padding='same')(encoder)
decoder = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder)
decoder = UpSampling2D((2, 2))(decoder)
output_layer = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoder)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, X_test))

# Build the classifier on top of the encoder
encoder_output = GlobalAveragePooling2D()(encoder)
classifier = Dense(2, activation='softmax')(encoder_output)

# Combine the autoencoder and classifier into a single model
model = Model(inputs=input_layer, outputs=classifier)
model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the combined model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy:.2f}")

Epoch 1/20
17/17 [==============================] - 47s 3s/step - loss: 0.0845 - val_loss: 0.0696
Epoch 2/20
17/17 [==============================] - 45s 3s/step - loss: 0.0458 - val_loss: 0.0144
Epoch 3/20
17/17 [==============================] - 67s 4s/step - loss: 0.0072 - val_loss: 0.0057
Epoch 4/20
17/17 [==============================] - 86s 5s/step - loss: 0.0040 - val_loss: 0.0029
Epoch 5/20
17/17 [==============================] - 90s 5s/step - loss: 0.0026 - val_loss: 0.0022
Epoch 6/20
17/17 [==============================] - 93s 5s/step - loss: 0.0021 - val_loss: 0.0018
Epoch 7/20
17/17 [==============================] - 94s 6s/step - loss: 0.0018 - val_loss: 0.0017
Epoch 8/20
17/17 [==============================] - 99s 6s/step - loss: 0.0017 - val_loss: 0.0016
Epoch 9/20
17/17 [==============================] - 90s 5s/step - loss: 0.0015 - val_loss: 0.0014
Epoch 10/20
17/17 [==============================] - 99s 6s/step - loss: 0.0013 - val_loss: 0.0013
Epoch 11/20
17/17 [

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\HP\anaconda\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\HP\anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\HP\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\HP\anaconda\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
      app.start()
    File "C:\Users\HP\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "C:\Users\HP\anaconda\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\HP\anaconda\lib\asyncio\base_events.py